In [1]:
import pandas as pd
import numpy as np
import logging
import time
import multiprocessing

import worker
import basicStrategies as bs

from functools import partial

from constants import *
from data import CSVDataHandler
from strategy import Indicator, Policy, Strategy, StrategyIterator


In [2]:
bars = CSVDataHandler('data', TIMEFRAME_DAY,
                      {'IMOEX':
                       {'src': 'finam', 'tz': 'Europe/Moscow'}
                      })
symbol = 'IMOEX'
df = bars.all_bars(symbol)

In [3]:
df.head()

,open,high,low,close,volume
datetime,,,,,
2000-01-05,173.00,173.00,173.00,173.00,0.0
2000-01-06,186.26,186.26,186.26,186.26,0.0
2000-01-10,200.81,200.81,200.81,200.81,0.0
2000-01-11,199.57,199.57,199.57,199.57,0.0
2000-01-12,196.88,196.88,196.88,196.88,0.0


In [4]:
PROCESSES = multiprocessing.cpu_count() - 1
result = {}

def error_handler(e):
    print('error: {}'.format(e))
    print('-->{}<--'.format(e.__cause__))

def run(res: dict, data: tpFrame, strat: Strategy):
    print(f'Running with {PROCESSES} processes...')
    start = time.time()
    with multiprocessing.Pool(PROCESSES) as p:
        result = p.map_async(partial(worker.calculate_all, data, strat), 
                             [ps for ps in StrategyIterator(strat)], 
                             callback=res.update, 
                             error_callback=error_handler).wait()
        #result.get(timeout=120)
        p.close()
        p.join()
    print(f'Time taken = {time.time() - start:.10f}')

if __name__ == '__main__':
    run(result, df, bs.turnoverExitRange)

Running with 7 processes...
Time taken = 30.6831514835


In [5]:
metrics = None
for k, v in result.items():
    if metrics is None:
        metrics = pd.DataFrame(columns = ['Feature', k])
        metrics.set_index('Feature', inplace = True)
    metrics[k] = v[k]

In [6]:
metrics

,"Strategy: Turnover and exit\nRunnning mode: all\nLong parameters: {'prices': 'close', 'window': 15}\nExit long parameters: {'window': 3}\nShort parameters: {'prices': 'close'}","Strategy: Turnover and exit\nRunnning mode: all\nLong parameters: {'prices': 'close', 'window': 15}\nExit long parameters: {'window': 4}\nShort parameters: {'prices': 'close'}","Strategy: Turnover and exit\nRunnning mode: all\nLong parameters: {'prices': 'close', 'window': 15}\nExit long parameters: {'window': 5}\nShort parameters: {'prices': 'close'}","Strategy: Turnover and exit\nRunnning mode: all\nLong parameters: {'prices': 'close', 'window': 15}\nExit long parameters: {'window': 6}\nShort parameters: {'prices': 'close'}","Strategy: Turnover and exit\nRunnning mode: all\nLong parameters: {'prices': 'close', 'window': 18}\nExit long parameters: {'window': 3}\nShort parameters: {'prices': 'close'}","Strategy: Turnover and exit\nRunnning mode: all\nLong parameters: {'prices': 'close', 'window': 18}\nExit long parameters: {'window': 4}\nShort parameters: {'prices': 'close'}","Strategy: Turnover and exit\nRunnning mode: all\nLong parameters: {'prices': 'close', 'window': 18}\nExit long parameters: {'window': 5}\nShort parameters: {'prices': 'close'}","Strategy: Turnover and exit\nRunnning mode: all\nLong parameters: {'prices': 'close', 'window': 18}\nExit long parameters: {'window': 6}\nShort parameters: {'prices': 'close'}","Strategy: Turnover and exit\nRunnning mode: all\nLong parameters: {'prices': 'close', 'window': 21}\nExit long parameters: {'window': 3}\nShort parameters: {'prices': 'close'}","Strategy: Turnover and exit\nRunnning mode: all\nLong parameters: {'prices': 'close', 'window': 21}\nExit long parameters: {'window': 4}\nShort parameters: {'prices': 'close'}",...,"Strategy: Turnover and exit\nRunnning mode: longs\nLong parameters: {'prices': 'open', 'window': 39}\nExit long parameters: {'window': 6}","Strategy: Turnover and exit\nRunnning mode: longs\nLong parameters: {'prices': 'open', 'window': 42}\nExit long parameters: {'window': 3}","Strategy: Turnover and exit\nRunnning mode: longs\nLong parameters: {'prices': 'open', 'window': 42}\nExit long parameters: {'window': 4}","Strategy: Turnover and exit\nRunnning mode: longs\nLong parameters: {'prices': 'open', 'window': 42}\nExit long parameters: {'window': 5}","Strategy: Turnover and exit\nRunnning mode: longs\nLong parameters: {'prices': 'open', 'window': 42}\nExit long parameters: {'window': 6}","Strategy: Turnover and exit\nRunnning mode: longs\nLong parameters: {'prices': 'open', 'window': 45}\nExit long parameters: {'window': 3}","Strategy: Turnover and exit\nRunnning mode: longs\nLong parameters: {'prices': 'open', 'window': 45}\nExit long parameters: {'window': 4}","Strategy: Turnover and exit\nRunnning mode: longs\nLong parameters: {'prices': 'open', 'window': 45}\nExit long parameters: {'window': 5}","Strategy: Turnover and exit\nRunnning mode: longs\nLong parameters: {'prices': 'open', 'window': 45}\nExit long parameters: {'window': 6}",Strategy: Turnover and exit\nRunnning mode: shorts\nShort parameters: {'prices': 'close'}
Feature,,,,,,,,,,,,,,,,,,,,,
Net Profit,874.63,1066.83,993.85,576.41,1050.12,1072.75,1182.1,1011.5,560.21,567.44,...,655.42,695.46,695.72,901.07,804.28,688.69,742.31,924.96,1023.11,182.06
Total Commission,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Number of Trades,362,341,326,313,336,317,304,290,312,298,...,145,150,143,136,131,148,142,136,133,174
Average Profit,2.42,3.13,3.05,1.84,3.13,3.38,3.89,3.49,1.8,1.9,...,4.52,4.64,4.87,6.63,6.14,4.65,5.23,6.8,7.69,1.05
"Average Profit, %",0.36,0.38,0.41,0.3,0.37,0.42,0.46,0.38,0.29,0.23,...,0.41,0.34,0.35,0.46,0.49,0.36,0.41,0.54,0.64,0.19
Average Bars Held,4.37,5,5.64,6.26,4.38,5,5.63,6.29,4.41,5,...,7,4,5,6,7,4,5,6,7,5
Winning Trades,190,183,178,167,178,175,165,154,162,158,...,80,82,81,77,74,84,84,78,79,83
"Win Rate, %",52.49,53.67,54.6,53.35,52.98,55.21,54.28,53.1,51.92,53.02,...,55.17,54.67,56.64,56.62,56.4